In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import trend_finder_lr as tflr
import trend_finder_ovr as tfovr
import ml_models as ml
import yfinance as yf
import pickle
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, svm
from imblearn.over_sampling import RandomOverSampler

# Run a test looking for stock price increases and decreases of percent, 30 days out
- Record 60 trading days before the trend
- use those 60 trading days as features

In [31]:
tesla = tf.Historydf('TSLA', 30, 5, 60)

In [32]:
tesla.df

,Open,Close,Volume
0,126.599335,125.806000,247117500
1,124.333336,125.235336,183079500
2,124.045334,133.455338,309177000
3,136.317337,136.665329,322344000
4,141.751999,134.279999,300954000
...,...,...,...
751,235.699997,239.759995,98595300
752,238.729996,232.960007,88197600
753,228.020004,225.600006,112484500
754,226.059998,219.220001,120237700


# Find trends, enter [Buy/Sell] classifier, and create new dataframe

In [33]:
tesla.find_trends()

start index = 59, end index = 726


In [35]:
tesla.pretrends_df

,Open,Close,Volume,Buy/Sell
0,214.119995,214.970001,98086569.0,NaN
1,126.599335,125.806000,247117500.0,NaN
2,124.333336,125.235336,183079500.0,NaN
3,124.045334,133.455338,309177000.0,NaN
4,136.317337,136.665329,322344000.0,NaN
...,...,...,...,...
33415,258.029999,257.500000,131283400.0,NaN
33416,260.600006,261.769989,112267600.0,NaN
33417,276.489990,279.820007,119685900.0,NaN
33418,278.820007,282.480011,131530900.0,NaN


# Structure the data in preperation for training model and normalize the values

In [36]:
tesla.structure_data()

In [37]:
tesla.structured_df

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,Buy/Sell
0,0.991897,0.157572,0.135971,0.133225,0.250213,0.302021,0.205809,0.259898,0.336454,0.409323,...,0.038709,0.009811,0.215022,0.075601,0.130693,0.107867,0.069332,0.000000,0.135667,1
1,0.288420,0.248880,0.243855,0.457989,0.552818,0.376711,0.475717,0.615844,0.749223,0.665387,...,0.009811,0.215022,0.075601,0.130693,0.107867,0.069332,0.000000,0.135667,0.088655,1
2,0.248880,0.243855,0.457989,0.552818,0.376711,0.475717,0.615844,0.749223,0.665387,1.000000,...,0.248781,0.115357,0.168079,0.146235,0.109357,0.043007,0.172839,0.127849,0.000000,1
3,0.243855,0.457989,0.552818,0.376711,0.475717,0.615844,0.749223,0.665387,1.000000,0.865352,...,0.115357,0.168079,0.146235,0.109357,0.043007,0.172839,0.127849,0.000000,0.024700,1
4,0.457989,0.552818,0.376711,0.475717,0.615844,0.749223,0.665387,1.000000,0.865352,0.447973,...,0.168079,0.146235,0.109357,0.043007,0.172839,0.127849,0.000000,0.024700,0.023870,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,0.364765,0.309250,0.248510,0.222875,0.277982,0.311046,0.247530,0.255613,0.274961,0.281737,...,0.639887,0.623992,1.000000,0.634288,0.713324,0.741057,0.618757,0.576444,0.344522,0
553,0.309250,0.248510,0.222875,0.277982,0.311046,0.247530,0.255613,0.274961,0.281737,0.216018,...,0.623992,1.000000,0.634288,0.713324,0.741057,0.618757,0.576444,0.344522,0.189711,0
554,0.239346,0.214656,0.267731,0.299575,0.238402,0.246186,0.264822,0.271348,0.208052,0.106385,...,1.000000,0.634288,0.713324,0.741057,0.618757,0.576444,0.344522,0.189711,0.250105,0
555,0.210259,0.262246,0.293438,0.233518,0.241143,0.259396,0.265789,0.203789,0.104205,0.093654,...,0.634288,0.713324,0.741057,0.618757,0.576444,0.344522,0.189711,0.250105,0.346537,0


# Run logistic regression

In [38]:
tesla_lr = ml.logistic_regression('Tesla')

In [39]:
tesla_lr.train_model(tesla.structured_df)

The testing accuracy is: 0.62
Coefficients:
 [[ 0.31476606 -0.09766818 -0.18871591  0.16632536  0.57363771 -0.745685
  -0.04629937 -0.35469253  0.34418044  0.03953948  0.29579623  0.17149206
  -0.15032408  0.01144455 -0.16326467 -0.28726923 -0.14194824  0.46614978
   0.25600698  0.06315962 -0.10438999 -0.19049212  0.00454959  0.57645033
   0.03332131  0.10588418  0.25113783  0.40992015 -0.07323598  0.18245079
   0.06883561  0.1261428   0.20957249 -0.18009581 -0.15041678  0.02177754
   0.11402885  0.41333828  0.43106949 -0.13655046 -0.09261732  0.1473362
  -0.28009766 -0.30740213 -0.43351991 -0.37623874 -0.0118674  -0.47129821
  -0.15088231 -0.14295074 -0.59908284  0.06766404 -0.43585372  0.02028604
  -0.26323314 -0.09289212 -0.08633409 -0.61530012  0.05324047  0.07378029
   0.35899785  0.3042472  -0.08227166  0.09977955 -0.41674175 -0.4957603
  -0.6101009   0.03484692 -0.54707278  0.09522722 -0.22192043 -0.46130805
  -0.10656784  0.06919608 -0.42856325 -0.19809409  0.07289678  0.398699

(LogisticRegression(solver='liblinear'), 0.6232876712328768)

# Adding a third classification for hold
- hold is represented by 2
- will include any points without increases or decreases of desired percentage
- model being used is One VS All

In [1]:
import trend_finder_ovr as ovr

In [39]:
tesla = ovr.Historydf('TSLA', '5y', 60, 5, 60)

In [40]:
tesla.find_trends()

start index = 59, end index = 1198


In [43]:
tesla.pretrends_df

,Open,Close,Volume,Buy/Sell
0,221.551193,228.438705,85935101.0,NaN
1,20.707333,21.459999,197583000.0,NaN
2,21.391333,21.442667,89190000.0,NaN
3,21.275999,21.340000,77209500.0,NaN
4,21.379999,21.521334,54039000.0,NaN
...,...,...,...,...
68335,168.410004,173.860001,125473600.0,NaN
68336,174.220001,176.889999,109520300.0,NaN
68337,177.169998,180.139999,136024200.0,NaN
68338,180.699997,188.869995,132001400.0,NaN


In [44]:
tesla.structure_data()

In [45]:
tesla.structured_df.head(20)

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,Buy/Sell
0,0.967471,0.018907,0.022138,0.021593,0.022084,0.021234,0.021363,0.018800,0.016278,0.016196,...,0.133847,0.146354,0.139960,0.140726,0.105178,0.125533,0.116088,0.049795,0.111122,0
1,0.595498,0.697243,0.680087,0.695557,0.668782,0.672848,0.592126,0.512693,0.510115,0.459936,...,0.146354,0.139960,0.140726,0.105178,0.125533,0.116088,0.049795,0.111122,0.061437,0
2,0.697243,0.680087,0.695557,0.668782,0.672848,0.592126,0.512693,0.510115,0.459936,0.342027,...,0.139960,0.140726,0.105178,0.125533,0.116088,0.049795,0.111122,0.061437,0.047848,0
3,0.680087,0.695557,0.668782,0.672848,0.592126,0.512693,0.510115,0.459936,0.342027,0.339548,...,0.140726,0.105178,0.125533,0.116088,0.049795,0.111122,0.061437,0.047848,0.034050,0
4,0.676048,0.650024,0.653976,0.575518,0.498313,0.495807,0.447036,0.332434,0.330024,0.091952,...,0.105178,0.125533,0.116088,0.049795,0.111122,0.061437,0.047848,0.034050,0.119932,0
5,0.637550,0.641425,0.564473,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,...,0.099177,0.089447,0.021157,0.084332,0.033149,0.019151,0.004937,0.093407,0.179210,0
6,0.641425,0.564473,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,...,0.089447,0.021157,0.084332,0.033149,0.019151,0.004937,0.093407,0.179210,0.120783,0
7,0.564473,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,...,0.021157,0.084332,0.033149,0.019151,0.004937,0.093407,0.179210,0.120783,0.007032,0
8,0.488750,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,...,0.093011,0.042313,0.028448,0.014368,0.102000,0.186990,0.129116,0.016443,0.000000,0
9,0.486292,0.438457,0.326054,0.323691,0.090187,0.214596,0.273303,0.291927,0.354131,0.361127,...,0.042313,0.028448,0.014368,0.102000,0.186990,0.129116,0.016443,0.000000,0.128688,0


In [81]:
from sklearn.multiclass import OneVsRestClassifier

In [83]:
tesla_lr = ml.one_vs_rest('Tesla')

In [84]:
tesla_lr.train_model(tesla.structured_df)

The testing accuracy is: 0.54


C:\Users\asbot\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\asbot\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

AttributeError: 'OneVsRestClassifier' object has no attribute 'coef_'